In [ ]:
import random

# --- Datos del problema ---
tasks = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
precedence = {
    'C': ['A'],
    'D': ['A'],
    'E': ['B'],
    'F': ['C', 'D'],
    'G': ['E', 'F'],
    'H': ['G'],
    'I': ['F'],
    'J': ['H', 'I']
}

def is_valid(order):
    position = {task: i for i, task in enumerate(order)}
    for task, preds in precedence.items():
        for pred in preds:
            if position[pred] > position[task]:
                return False
    return True

def fitness(order):
    position = {task: i for i, task in enumerate(order)}
    score = 100
    for task, preds in precedence.items():
        for pred in preds:
            if position[pred] > position[task]:
                score -= 30
            else:
                # Penalizar distancia grande entre pred y task
                score -= (position[task] - position[pred] - 1) * 2
    return score


## Generar población

In [20]:
def create_individual():
    return random.sample(tasks, len(tasks))

def create_population(size=20):
    return [create_individual() for _ in range(size)]

population = create_population()

for ind in population:
    print(ind, "-> Fitness:", fitness(ind))


['J', 'H', 'D', 'C', 'I', 'G', 'F', 'B', 'E', 'A'] -> Fitness: -150
['B', 'G', 'J', 'D', 'C', 'H', 'A', 'E', 'I', 'F'] -> Fitness: -146
['G', 'A', 'I', 'E', 'H', 'J', 'C', 'F', 'B', 'D'] -> Fitness: -82
['F', 'D', 'B', 'G', 'I', 'E', 'H', 'J', 'A', 'C'] -> Fitness: -42
['I', 'B', 'F', 'A', 'G', 'J', 'C', 'E', 'D', 'H'] -> Fitness: -90
['C', 'B', 'J', 'G', 'D', 'F', 'A', 'E', 'I', 'H'] -> Fitness: -112
['A', 'I', 'E', 'B', 'G', 'J', 'F', 'C', 'H', 'D'] -> Fitness: -122
['C', 'F', 'I', 'H', 'E', 'B', 'A', 'J', 'D', 'G'] -> Fitness: -58
['J', 'C', 'D', 'A', 'B', 'H', 'I', 'F', 'G', 'E'] -> Fitness: -136
['C', 'H', 'I', 'G', 'B', 'E', 'J', 'D', 'A', 'F'] -> Fitness: -112
['B', 'I', 'C', 'H', 'F', 'A', 'E', 'D', 'G', 'J'] -> Fitness: -66
['F', 'I', 'B', 'H', 'G', 'E', 'D', 'C', 'A', 'J'] -> Fitness: -114
['A', 'C', 'G', 'D', 'F', 'I', 'E', 'B', 'H', 'J'] -> Fitness: -14
['A', 'E', 'F', 'B', 'I', 'D', 'J', 'G', 'C', 'H'] -> Fitness: -66
['A', 'G', 'C', 'B', 'D', 'E', 'J', 'I', 'H', 'F'] -> F

## Operadores GA: selección, cruce, mutación

In [3]:
def selection(pop):
    a, b = random.sample(pop, 2)
    return a if fitness(a) > fitness(b) else b

def crossover(p1, p2):
    size = len(p1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None]*size
    child[start:end] = p1[start:end]
    fill = [x for x in p2 if x not in child]
    idx = 0
    for i in range(size):
        if child[i] is None:
            child[i] = fill[idx]
            idx += 1
    return child

def mutate(ind, prob=0.2):
    if random.random() < prob:
        i, j = random.sample(range(len(ind)), 2)
        ind[i], ind[j] = ind[j], ind[i]
    return ind


## Evolución

In [21]:
def evolve(population, generations=20):
    for gen in range(generations):
        new_pop = []
        for _ in range(len(population)):
            p1 = selection(population)
            p2 = selection(population)
            child = crossover(p1, p2)
            child = mutate(child)
            new_pop.append(child)
        population = new_pop
        best = max(population, key=fitness)
        print(f"Gen {gen+1:2d}: {best} | Fitness: {fitness(best)} | Válido: {is_valid(best)}")
    return best

best = evolve(population)
print("\nMejor solución encontrada:", best)


Gen  1: ['D', 'F', 'B', 'G', 'I', 'E', 'H', 'J', 'A', 'C'] | Fitness: -8 | Válido: False
Gen  2: ['B', 'H', 'A', 'D', 'F', 'I', 'E', 'J', 'C', 'G'] | Fitness: -4 | Válido: False
Gen  3: ['A', 'D', 'F', 'I', 'B', 'E', 'H', 'J', 'C', 'G'] | Fitness: 2 | Válido: False
Gen  4: ['A', 'D', 'F', 'I', 'B', 'J', 'E', 'C', 'G', 'H'] | Fitness: 12 | Válido: False
Gen  5: ['D', 'F', 'I', 'B', 'E', 'G', 'H', 'J', 'A', 'C'] | Fitness: 26 | Válido: False
Gen  6: ['B', 'E', 'F', 'A', 'I', 'C', 'G', 'H', 'J', 'D'] | Fitness: 6 | Válido: False
Gen  7: ['D', 'F', 'I', 'B', 'E', 'G', 'H', 'A', 'C', 'J'] | Fitness: 18 | Válido: False
Gen  8: ['D', 'B', 'C', 'F', 'E', 'I', 'G', 'H', 'J', 'A'] | Fitness: 20 | Válido: False
Gen  9: ['E', 'A', 'B', 'C', 'D', 'F', 'I', 'G', 'H', 'J'] | Fitness: 44 | Válido: False
Gen 10: ['A', 'B', 'C', 'D', 'F', 'H', 'E', 'G', 'I', 'J'] | Fitness: 38 | Válido: False
Gen 11: ['A', 'B', 'C', 'D', 'F', 'E', 'G', 'H', 'I', 'J'] | Fitness: 76 | Válido: True
Gen 12: ['A', 'C', 'B', 